In [0]:
!pip install -U pytorch-pretrained-bert

     |████████████████████████████████| 133kB 2.8MB/s 


In [0]:
import torch
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [0]:
class FillBlanks:
  def __init__(self, text):
    self.text = text

  def __prep_text(self):
    return "[CLS] " + self.text.replace("__", "[MASK]") + " [SEP]"
  
  def __result_text(self, predicted_token):
    return self.text.replace("__", predicted_token)

  def fill_up(self):

    prepped_text = self.__prep_text()

    tokenized_text = tokenizer.tokenize(prepped_text)
    masked_index = tokenized_text.index('[MASK]')
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)

    # Create the segments tensors.
    segments_ids = [0] * len(tokenized_text)

    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])

    # Load pre-trained model (weights)
    model = BertForMaskedLM.from_pretrained('bert-base-uncased')
    model.eval()

    # Predict all tokens
    with torch.no_grad():
        predictions = model(tokens_tensor, segments_tensors)

    predicted_index = torch.argmax(predictions[0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]

    print("Predicted Word: " + predicted_token)
    print(self.__result_text(predicted_token))

In [0]:
# Gap must be consists of "__" (Without quotes)

text = "Damn, Tesla model 3 is so awesome that i really __ it."
f1 = FillBlanks(text)
f1.fill_up()

Predicted Word: love
Damn, Tesla model 3 is so awesome that i really love it.
